## Generate MATLAB Grader rosters for the quiz
Simeon Wong

In [ ]:
# imports
import pandas as pd
import time

In [ ]:
# import and merge data
grades = pd.read_csv('Grades.csv')
roster = pd.read_csv('Roster.csv')

grades = grades.merge(roster[['UTORid', 'Email']], left_on='SIS User ID', right_on='UTORid', how='left')
grades['Integration ID'] = grades['Integration ID'].apply(lambda x: '{:.0f}'.format(x))

In [ ]:
# subset the information we need later on
class_list = grades[['Email', 'Section', 'UTORid', 'Integration ID']].iloc[2:,:]
class_list.rename(columns={'Integration ID': 'StudentNum'}, inplace=True)

# parse practicals section
class_list['Practical'] = class_list['Section'].str.extract(r'F-(PRA[0-9]{4})-2022')
class_list.drop('Section', axis=1, inplace=True)

# save a copy for attendance
attendance_list = class_list.copy()
class_list

In [ ]:
# lookup practical timing
times = pd.read_csv('PracticalTiming.csv', sep='\t')
times['Section'] = times['Section'].str[:7]
times['Day'] = times['Day'].str[:1]
times['Time'] = times['Time'].apply(lambda x: time.strptime(x, '%I:%M %p').tm_hour)
times['String'] = times.apply(lambda x: str(x['Day']) + '.' + str(x['Time']), axis=1)
times

In [ ]:
# Merge practical timing with class list
class_list = class_list.merge(times[['Section', 'String']], left_on='Practical', right_on='Section', how='left')
class_list

In [ ]:
# Generate a list of unique time slots
time_list = class_list['String'].dropna().unique()
time_list.sort()

# Generate a list of students within each time slot
with open('GraderInvites.txt', 'w') as f:
    for tl in time_list:
        f.writelines('\n\n------------------------\nMAT188 2022F - QUIZ {}\n\n------------------------\n'.format(tl))
        subset = class_list[class_list['String'] == tl]['Email']
        subset = subset.dropna()
        f.write(', '.join(subset.tolist()))
        f.write('\n\n')

# Show the list of students without emails so we can look into it
class_list.loc[class_list['Email'].isna(), :]

In [ ]:
# Write attendance lists to file for each practical section for TAs to take quiz attendance
sections = attendance_list['Practical'].dropna().unique()
sections.sort()

attendance_list['Present?'] = ''
attendance_list['Time out'] = ''
attendance_list['Comments'] = ''

with pd.ExcelWriter('QuizAttendanceList.xlsx') as xlsw:
    for s in sections:
        subset = attendance_list[attendance_list['Practical'] == s]
        subset.to_excel(xlsw, sheet_name=s, index=False)
